In [1]:
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import numpy as np

In [2]:
#запуск и настройка драйвера
def start_driver(url):
    options = Options()
    options.add_argument('--disable-blink-features=AutomationControlled') #без этого аргумента блочит соединение
    options.add_argument("--incognito")
    driver = webdriver.Chrome(options=options)
    driver.execute_cdp_cmd("Page.removeScriptToEvaluateOnNewDocument", {"identifier":"1"}) #без этой команды блочит соединение
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html)
    return driver, soup

#сбор ссылок всех товаров категории
def get_href(base_url, category, driver, soup):
    href=[]

    for i in range(100):
        for div in soup.find_all('div',class_="product-name"):
            href.append(div.a['href'])
        time.sleep(2)
        
        url = base_url+category+str(i+2)
        driver.get(url)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html)
        
        if soup.find('div',class_="not-found") is not None:
            break
        
    return href

#сбор данных из ссылок на товар
def get_parsed_data(df, base_url, category):
    url = base_url+category+str(1)
    driver, soup = start_driver(url)

    df['Ссылка на товар'] = get_href(base_url, category, driver, soup)

    product_name = []
    discount_cost = []
    online_cost = []
    weight = []
    
    for i in range(df.shape[0]):
        url = base_url + df['Ссылка на товар'][i]
        driver.get(url)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html)
        
        product_name.append(soup.find('h1',class_='text-title-4 font-normal mb-2 text-left').get_text(strip=True))
        
        try:
            discount_cost.append(int(soup.find('div',class_='text-title-3 mb-2').get_text(strip=True).replace(u'\xa0', u' ').replace(' ₽', '').replace(' ', '')))
        except: #для товаров без указанной цены со скидкой
            discount_cost.append(np.nan)
        
        try:
            online_cost.append(int(soup.find('span',class_='text-body-2 font-semibold amount').get_text(strip=True).replace(u'\xa0', u' ').replace(' ₽', '').replace(' ', '')))
        except: #для товаров без указанной цены online
            online_cost.append(np.nan)
        
        try: 
            weight.append(float(soup.find_all('div', class_='value')[1].find_all('span')[2].get_text(strip=True).replace('вес от ', '').replace(' гр', '')))
        except: #для товаров с блоком Вставки, который смещает блок Материалы на +1
            try:
                weight.append(float(soup.find_all('div', class_='value')[2].find_all('span')[2].get_text(strip=True).replace('вес от ', '').replace(' гр', '')))
            except: #для товаров без указанного веса
                weight.append(np.nan)
        time.sleep(2)
        
        if i % 100 == 0: #очистка памяти браузера
            driver.quit()
            url = base_url + df['Ссылка на товар'][i]
            driver, soup = start_driver(url)
            
    df['Название товара'] = product_name
    df['Цена после скидок'] = discount_cost
    df['Цена при оплате online'] = online_cost
    df['Вес изделия'] = weight
    
    return df

In [3]:
df = pd.DataFrame(columns=['Название товара', 'Цена после скидок', 'Цена при оплате online', 'Вес изделия', 'Ссылка на товар'])

base_url = "https://www.585zolotoy.ru"
category = "/catalog/gold-bracelets-with-stones/?page="

parsed_data = get_parsed_data(df, base_url, category)
parsed_data.to_excel('parsed_data.xlsx')
parsed_data

,Название товара,Цена после скидок,Цена при оплате online,Вес изделия,Ссылка на товар
0,Серебряный браслет с ониксом и родированием,4740.0,4503.0,3.24,/catalog/products/1595222/
1,Серебряный браслет с фианитами и родированием ...,1960.0,1862.0,1.87,/catalog/products/1754148/
2,Серебряный браслет на ногу Месяц с фианитами и...,1640.0,1558.0,1.78,/catalog/products/1233799/
3,Серебряный браслет с фианитами и родированием ...,1240.0,1178.0,0.97,/catalog/products/1357337/
4,Серебряный браслет с кварцем и позолотой,NaN,NaN,NaN,/catalog/products/1332308/
...,...,...,...,...,...
702,Серебряный браслет с малахитом и родированием,1500.0,NaN,NaN,/catalog/products/2216048/
703,Серебряный браслет с эмалью и жемчугом р. 16-17,4680.0,4446.0,NaN,/catalog/products/8152463/
704,Серебряный браслет с фианитами и родированием,800.0,NaN,NaN,/catalog/products/6568881/
705,Серебряный браслет с турмалинами и фианитами с...,5840.0,5548.0,NaN,/catalog/products/7420537/
